# GFDL FLOR Historical

## Loads in all monthly forecasts of sea ice extent
## Regrids to polar stereographic,
## Saves to a netcdf file

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import xarray as xr
import xesmf as xe
import os
import glob
import seaborn as sns

In [ ]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [ ]:
# Data dir
data_dir = r'/home/disk/sipn/nicway/data/model/nmme/GFDL_FLORB01/hist/nomads.gfdl.noaa.gov'
data_out = r'/home/disk/sipn/nicway/data/model/nmme/GFDL_FLORB01/hist/stereo'

In [ ]:
def preprocess_time(x):
    ''' Convert time to initialization and foreast lead time (to fit into orthoganal matrices)'''
    # Get initialization time
    x.coords['init_time'] = xr.decode_cf(x).isel(time=0).time.values;
    #     x.coords['init_time'].attributes
    # Get forecast time in days from initilization
    x.coords['time'] = np.arange(1,13,1)*30 - 15 # x.time.values.astype('timedelta64[D]').astype(int)
    # Rename and decode
    return x.rename({'time':'fore_time'}, inplace=True)

In [ ]:
# Open a single ensemble member
def open_1_ensemble(cfiles, e):
    ds = xr.open_mfdataset(cfiles, concat_dim='init_time', decode_times=False, 
                           preprocess=lambda x: preprocess_time(x),
                           autoclose=True)
    ds.coords['ensemble'] = e
    return ds

In [ ]:
# Open all ensemble members as a dataset
ds_list = []
for e in np.arange(1,13,1):
    ensm_name = 'r'+str(e)+'i1p1'
    cfiles = os.path.join(data_dir, '*sic*'+ensm_name+'*.nc')
    ds_list.append(open_1_ensemble(cfiles, e))
ds_all = xr.concat(ds_list, dim='ensemble')

In [ ]:
ds_list = None

In [ ]:
# Average over all sea ice categories 
sic = ds_all.sic.sum(dim='ct', skipna=False)

In [ ]:
# Get info about target grid
flat = r'/home/disk/sipn/nicway/data/grids/psn25lats_v3.dat'
flon = r'/home/disk/sipn/nicway/data/grids/psn25lons_v3.dat'
NY=304; 
NX=448;

def readBinFile(f, nx, ny):
    with open(f, 'rb') as fid:
        data_array = np.fromfile(f, np.int32)*1e-5
    return data_array.reshape((nx,ny))

In [ ]:
lat = readBinFile(flat, NX, NY).T
lon = readBinFile(flon, NX, NY).T
ds_out = xr.Dataset({'lat': (['x', 'y'],  lat), 'lon': (['x', 'y'], lon)})
ds_out

In [ ]:
# Plot target lat and lon
# ds_out.lon.plot.pcolormesh()

In [ ]:
# ds_out.lat.plot.pcolormesh()

In [ ]:
# Calculate regridding matrix
regridder = xe.Regridder(sic, ds_out, 'bilinear', periodic=True)
# Regrid
sic_out = regridder(sic)
# sic_out

In [ ]:
sic_out.dims

In [ ]:
# Define naive_fast that searches for the nearest WRF grid cell center
def naive_fast(latvar,lonvar,lat0,lon0):
    # Read latitude and longitude from file into numpy arrays
    latvals = latvar[:]
    lonvals = lonvar[:]
    dist_sq = (latvals-lat0)**2 + (lonvals-lon0)**2
    minindex_flattened = dist_sq.argmin()  # 1D index of min element
    iy_min,ix_min = np.unravel_index(minindex_flattened, latvals.shape)
    return iy_min,ix_min

In [ ]:
# Set up plotting info
cmap_sic = matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10))
cmap_sic.set_bad(color = 'lightgrey')

In [ ]:
# Plot original projection
plt.figure(figsize=(20,10))
ax1 = plt.axes(projection=ccrs.PlateCarree())
# ds_p = sic.mean(dim='init_time').mean(dim='fore_time').mean(dim='ensemble')
ds_p = sic.isel(init_time=0).isel(fore_time=6).mean(dim='ensemble')
ds_p.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                 vmin=0, vmax=1,
                                 cmap=matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10)));
ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
gl = ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
gl.xlabels_bottom = True
gl.ylabels_left = True
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER
ax1.coastlines(linewidth=0.75, color='black', resolution='50m');

In [ ]:
# Plot SIC on target projection
plt.figure(figsize=(10,8));
ax1 = plt.axes(projection=ccrs.NorthPolarStereo(central_longitude=0))
# ds_p2 = sic_out.mean(dim='init_time').mean(dim='fore_time').mean(dim='ensemble')
ds_p2 = sic_out.isel(init_time=0).isel(fore_time=6).mean(dim='ensemble')
ds_p2.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic)
# Add coastlines and meridians/parallels 
ax1.coastlines(linewidth=0.75, color='black', resolution='50m')
ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
ax1.set_extent([-180, 180, 50, 90], crs=ccrs.PlateCarree())
plt.tight_layout()

In [ ]:
# # Save regridded to netcdf file
f_out = os.path.join(data_out, 'GFDL_FLORB01_Stereo.nc')
sic_out.to_netcdf(f_out)

In [ ]:
# Clean up
regridder.clean_weight_file()  # clean-up